#The problem

**Question:** Find how many numbers exist such that the number is at most 16 digits, and has *at least* 4 prime factors, where the primes are less than 100.

So let's think about that. Here are some things to keep in mind:

- We have to find **how many** numbers like that there are, not the numbers themselves. Computing the numbers themselves will be too computationally expensive.
- For a number to be at most 16 digits, it has to be less than `10**16`.
- If we have a number which has 4 prime factors, then multiples of that number will also have the same factors. (This is simply a property of factoring. If 2 is a factor of 4, it is also a factor of all the multiples of 4, namely: 4, 8, 12, 16...)

Let's write some setup code.

In [1]:
import math
import itertools

count = 0
limit = 10**16 - 1 # No qualifying number should be above this limit
prime_limit = 100 # No prime factor should be above this limit
primes = [] # Used to store all the primes smaller than the limit

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
       if n % i == 0:
         return False
    return True


def get_primes(p_limit):
    for i in range(2, p_limit):
       if is_prime(i):
         primes.append(i) # add it to the global list we have

def lcm(a, b):
    small = min(a, b)
    # We start at the smaller number, and take steps of that size since the LCM has to be a multiple of this number
    # Given that, we know that the biggest LCM we can find is simply a * b, which is why that is the
    # end of the range
    for i in range(small, a * b + 1, small):
       if i % b == 0 and i % a == 0:
         return i


Given that code, which should be fairly simple to understand, let's just test it and make sure we get values that we expect out of it

In [2]:
assert is_prime(5)
assert is_prime(101)
assert is_prime(1200) == False
assert is_prime(1) == False

assert lcm(3, 6) == 6
assert lcm(3, 4) == 12
assert lcm(5, 7) == 35
assert lcm(6, 9) == 18

Alright, our basic helper functions work. Let's try to figure the problem out now. The key thing to understand is that if a `N` number has `a, b, c, d` as prime factors, then `2 * N` will also have the same primes as factors. So will `3 * N`. We can follow the pattern to get as many numbers as we want which have `a, b, c, d` as prime factors.

However, we have a condition to satisfy. These numbers cannot be more than 16 digits. So we need to know how many multiples of `N` there are, that are less than `limit`. If you think about this hard enough, you'll see that the answer is simply `limit // N` where `//` represents integer division.

For example, how many multiples of 4 are there below 10?

In [3]:
10 // 4 # Gives 2, because the list is: 4, 8

2

There is a problem, here. The following code should illustrate that:

In [4]:
12 // 4

3

That gives us 3, which means we're finding how many multiples of 4 there are that are less than *or equal* to 12. Fixing it is simple. We just run `11 // 4` instead. You'll note that this is why we used `limit = 10**16 - 1` in our starter code.

Anyway, let's get back to the problem. Given `N`, we can find how many multiples of N there are under the limit. Using this fact, we can simply find all the possible `N`'s we can form with our primes, and then we can just add `limit // current_N` to the count. Let's get around to doing that.

First, we'll get our list of primes.

In [9]:
get_primes(prime_limit)
print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


Let's see how we can use `itertools.combinations` to create unique combinations of sequences. The following line of code *chooses* 2 elements out of the list `[1, 2, 3]` to create the appropriate unique combinations.

In [10]:
print(list(itertools.combinations([1,2,3], 2)))

[(1, 2), (1, 3), (2, 3)]


Similarly, we can create a list of all the possible ways there are to combine 4 primes. For example:

In [12]:
print(list(itertools.combinations([3, 5, 7, 11], 2)))

[(3, 5), (3, 7), (3, 11), (5, 7), (5, 11), (7, 11)]


If we multiply the elements of each tuple in the list above, we'll get a list of numbers which have 2 prime factors. This will be the list of `N`s that we required. Let's write code to do that.

In [17]:
prime_combinations = itertools.combinations(primes, 4)
prime_multiples = [a * b * c * d for (a, b, c, d) in prime_combinations]

for N in prime_multiples:
    count = count + limit // N

print(count)

91948057480907778


Right now, `count` feels like an arbitrary number, and it may seem like we're done. However, there is more! We need to consider all the different N that have some common multiples.

For example, if one `N` is 5, we count all of its multiples: 5, 10, **15**, 20...

If another `N` is 3, we count: 3, 6, 9, 12, **15**, 18...

This means that we're overcounting certain numbers, since they're multiples of different N. *And we're overcounting the multiples of these certain numbers too.* In this example, the number 30 is also counted in both lists. So is 45. This means that we need to find a list of the least common multiples between various `N` and we need to subtract them and their multiples from our `count`.

